### Step 1. 데이터 읽어오기

In [1]:
import glob
import os
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf    # 대망의 텐서플로우!

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


### Step 2. 데이터 정제
나만의 방법으로 데이터 정제를 해보았다.

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    #sentence.split()
    
    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
#print(len(preprocess_sentence("This @_is ;;;sample        sentence."))
type(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


str

#### Step 2_1. 방법1 
 - *preprocess_sentence1*은 split 후에 if - else 를 사용하여 토큰 개수를 세서 14 초과이면 14까지 슬라이싱해서 넣고 14이하이면 그냥 넣는 방법이다. 하지만 이 방법으로 데이터 정제를 한 후에 데이터 수가 너무 많아서 학습하는데 시간이 오래걸리고, 긴 문장은 제외하라는 과제에 적합하지 않아 사용하지 않았다.

In [3]:
def preprocess_sentence1(sentence):

    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence=re.split(" ", sentence)
    
    if len(sentence) > 14 : short_sentence = sentence[:14]
        
    else : short_sentence = sentence
    
    short_sentence = " ".join(short_sentence)
    short_sentence = '<start> ' + short_sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return short_sentence

print(preprocess_sentence1("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
#print(len(preprocess_sentence("This @_is ;;;sample        sentence."))
type(preprocess_sentence1("This @_is ;;;sample        sentence."))

<start> this is sample sentence .  <end>


str

#### Step 2_2. 방법2 
- *preprocess_sentence2* 은 문장을 *split* 후에 *if - else* 를 사용하여 토큰 개수를 세서 14 이상이면 *return False* 를 하고 *else* 이면 *.join* 함수를 써서 *split* 한 것을 붙인 다음 문장을 *return* 한다. *return False* 한 값은 아래 코드 *for - if* 문에서 다시 한번 확인한다.

In [4]:
def preprocess_sentence2(sentence):

    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    
    sentence=re.split(" ", sentence)
    
    if len(sentence) > 13 : 
        return False
    
    else : 
        sentence = " ".join(sentence)
        sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
        return sentence

print(preprocess_sentence2("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.
#print(len(preprocess_sentence("This @_is ;;;sample        sentence."))
type(preprocess_sentence2("This @_is ;;;sample        sentence."))

<start> this is sample sentence .  <end>


str

#### Step 2_3. 방법2
 - *for - if* 문으로 *preprocess_sentence2* 에서 *return* 값이 *False* 이면 *continue* 로 제외시킨다.
 - tokenize 전에 정제된 데이터 수를 확인한다.
 - 정제된 데이터 수가 너무 적거나 많으면 다른 방법을 생각한다.

In [5]:
corpus = []

for sentence in raw_corpus:
    #print(len(sentence))
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if preprocess_sentence2(sentence) == False : continue
    #print(len(sentence))
    corpus.append(preprocess_sentence2(sentence))
        
print(corpus[:10])
print(type(corpus))
print(len(corpus))

['<start> there must be some kind of way outta here <end>', '<start> said the joker to the thief <end>', '<start> there s too much confusion <end>', '<start> i can t get no relief business men , they drink my wine <end>', '<start> plowman dig my earth <end>', '<start> none were level on the mind <end>', '<start> nobody up at his word <end>', '<start> hey , hey no reason to get excited <end>', '<start> the thief he kindly spoke <end>', '<start> there are many here among us <end>']
<class 'list'>
154531


#### Step 2_4. tokenize 
- *tokenize* 의 *num_words* 는 *14000* 으로 했다. *tensor*, *tokenizer* 를 출력하여 확인해보니 단어가 숫자화 되고 끝 부분은 0으로 *padding* 된 것을 확인할 수 있다.

In [6]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,sep='\n')
    print(tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   61  272 ...    0    0    0]
 [   2  119    6 ...    0    0    0]
 [   2   61   17 ...    0    0    0]
 ...
 [   2    6  583 ...    0    0    0]
 [   2    7  214 ...    0    0    0]
 [   2    8 3374 ...    0    0    0]]


#### Step 2_5. tokenize 값 확인 
- 텐서화 된 값을 자세히 출력해보면 이와같이 앞부분엔 무조건  *start* 가 있어 2 인 것을 확인할 수 있다. 그리고 문장 끝엔 3으로  *end* 가 들어간 것을 확인할 수 있다.

In [7]:
print(tensor[:3, :10])

[[   2   61  272   27   94  551   20   86  750   90]
 [   2  119    6 6225   10    6 2289    3    0    0]
 [   2   61   17  102  187 2688    3    0    0    0]]


In [8]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


##### 총 데이터셋 행렬 모양은 (154531, 14) 이고 src_input 과 tgt_input 모양이 같다는 것을 확인했다. 
- 문장 수 : 154531
- 문장의 최대 단어(토큰) 개수 : 14개

In [9]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])
print(len(src_input))
print(len(tgt_input))
print(type(src_input))
print(src_input.shape)

[  2  61 272  27  94 551  20  86 750  90   3   0   0   0]
[ 61 272  27  94 551  20  86 750  90   3   0   0   0   0]
154531
154531
<class 'numpy.ndarray'>
(154531, 14)


### Step 3. 평가 데이터셋 분리
- tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리했다.단어장의 크기는 14,000이고 총 데이터의 20%를 평가 데이터셋으로 사용했다.

In [10]:
from sklearn.model_selection import train_test_split

enc_train, enc_val = train_test_split(src_input, 
                                        test_size=0.2,
                                        random_state=34,
                                        shuffle=True)
dec_train, dec_val = train_test_split(tgt_input, 
                                        test_size=0.2,
                                        random_state=34,
                                        shuffle=True)

### Step 3-1. 평가 데이터셋 갯수 확인
- 데이터셋을 분리했을 때, 학습 데이터 갯수가 과제 목표 124960보다 작은 것을 확인했다.

In [ ]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

### Step 3-2. BUFFER_SIZE, dataset, val_dataset 설정 
- *BUFFER_SIZE* , *dataset* , *val_dataset* 에 분리한 데이터 셋 값들을 넣어주었다.

In [12]:
BUFFER_SIZE = len(enc_train)
print(len(enc_train))
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE
print(steps_per_epoch)
VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(len(enc_val))
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)


123624
482


### Step 4. 인공지능 만들기
- RNN을 사용했다. hidden_size를 2048로 높게 주었다. 임베딩을 통해 단어들간의 상대적인 거리 값으로 학습한다.

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)\
#print("f")

### Step 4_1. 셈플확인
- 최종 출력 텐서 shape가 shape=(256, 14, 14001)인 것을 확인 했다. 14001은 Dense 레이어의 출력 차원 수 이다. 14001개의 단어 중 어느 단어의 확률이 가장 높을지를 모델링해야 하기 때문이다. 256은 배치 사이즈이다. 

In [14]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 14001), dtype=float32, numpy=
array([[[ 7.3888616e-05,  4.1382926e-04, -1.3355503e-04, ...,
          2.4013220e-04, -1.6636026e-04,  3.5196939e-05],
        [ 1.8130390e-04,  6.0105446e-04, -2.2397129e-04, ...,
         -3.2661126e-05, -4.3243633e-04,  1.4015993e-04],
        [ 3.1436139e-04,  8.2724454e-04,  4.0651063e-05, ...,
         -2.1022202e-04, -6.4477813e-04,  1.5606832e-04],
        ...,
        [ 3.4853735e-04,  4.1987450e-04, -1.2359800e-04, ...,
         -2.5477511e-04,  8.6093909e-04, -4.1772844e-04],
        [ 3.9464838e-04,  7.7889890e-05, -2.5878238e-04, ...,
          1.6956119e-04,  9.0672815e-04, -4.5339690e-04],
        [ 4.8283782e-04, -2.8222139e-04, -3.6002140e-04, ...,
          6.0202961e-04,  8.8616187e-04, -5.1320856e-04]],

       [[ 7.3888616e-05,  4.1382926e-04, -1.3355503e-04, ...,
          2.4013220e-04, -1.6636026e-04,  3.5196939e-05],
        [-1.2775954e-04,  4.6104289e-04,  5.2953732e-05, ...,
          7.0682674e-04, 

In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3584256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  28688049  
Total params: 84,717,489
Trainable params: 84,717,489
Non-trainable params: 0
_________________________________________________________________


### Step 4_2. model.fit
- 만들어둔 dataset과 val_dataset 변수를 넣어주고 epoch을 10으로 하여 fit 해주었다.

In [16]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
#model.fit(dataset, epochs=10, validation_data = val_dataset ,steps_per_epoch = steps_per_epoch, validation_steps=len(dec_val) // BATCH_SIZE )
model.fit(dataset, epochs=10, validation_data = val_dataset , verbose=1 )

Epoch 1/10
482/482 [==============================] - 214s 445ms/step - loss: 3.4614 - val_loss: 3.0340
Epoch 2/10
482/482 [==============================] - 216s 449ms/step - loss: 2.8494 - val_loss: 2.7582
Epoch 3/10
482/482 [==============================] - 216s 449ms/step - loss: 2.5453 - val_loss: 2.5826
Epoch 4/10
482/482 [==============================] - 216s 448ms/step - loss: 2.2619 - val_loss: 2.4511
Epoch 5/10
482/482 [==============================] - 216s 449ms/step - loss: 1.9966 - val_loss: 2.3524
Epoch 6/10
482/482 [==============================] - 216s 449ms/step - loss: 1.7533 - val_loss: 2.2795
Epoch 7/10
482/482 [==============================] - 211s 438ms/step - loss: 1.5392 - val_loss: 2.2299
Epoch 8/10
482/482 [==============================] - 211s 437ms/step - loss: 1.3593 - val_loss: 2.2049
Epoch 9/10
482/482 [==============================] - 215s 447ms/step - loss: 1.2165 - val_loss: 2.1962
Epoch 10/10
482/482 [==============================] - 216s 447m

### Step 5. 인공지능 Test

In [17]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [18]:
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love ma little nasty girl <end> '

In [19]:
generate_text(model, tokenizer, init_sentence="<start> turn up")

'<start> turn up the bass til it s up in your face level <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> flex")

'<start> flex in the morning <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> I make it")

'<start> i make it spin on my finger im a critical thinker <end> '

In [23]:
generate_text(model, tokenizer, init_sentence="<start> im rockin")

'<start> im rockin the flow i pop em the other day <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> so easy")

'<start> so easy to say <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> god")

'<start> god damn , god damn <end> '

# 마치며 ...
 - 처음에 maxlen을 안쓰고 토큰화 하는 방법을 고민하다가 split, join, if-else-continue 방법으로 잘 되어 기뻤다. 하지만 val_loss가 잘 나와도 2.4정도까지 밖에 안내려가서 epochs도 낮춰보고, steps_per_epoch, validation_steps 값도 넣어보고, embedding_size, hidden_size 값을 낮추어 봤는데 계속 2.4 이상이 나왔다. 그래서 hidden_size 값을 2048로 대폭 올렸더니 val_loss가 2.2 이하가 나와서 기뻤다. 하지만 hidden_size 값을 2048로 주면 학습시간이 엄청 늘어났다. hidden_size = 1024 학습시간은 20분 정도 걸렸던거 같은데 hidden_size = 2048 학습시간은 1시간 정도 걸린 것 같다.   
---
- 노래 가사 출력은 오버피팅 되지 않고 적절하게 출력된 것 같다. 힙합을 좋아해서 힙합에서 낳오는 가사들을 몇개 넣어봤는데 만족스럽다. 근데 사실 val_loss가 2.4(hidden_size = 1024) 인 것과 2.2(hidden_size = 2048) 인 것의 출력 가사는 큰 차이는 없는 것 같다. memory 오류가 두려워 코드를 추가하지는 않겠다. 

평가 문항	상세 기준
1 . 가사 텍스트 생성 모델이 동작 하는가?

텍스트 제너레이션 결과가 그럴듯한 문장으로 생성 되는가?
2 . 데이터의 전처리와 데이터 셋 구성 과정이 체계적으로 진행 되었습니까?

특수 문자 제거, 토크 나이저 생성, 패딩 처리 등의 과정이 빠짐없이 진행 되었습니까?
3 . 텍스트 생성 모델이 안정적으로 학습 되었습니까?

텍스트 생성 모델의 유효성 검사 손실 2.2 이하로 생성 는가?